# Hybrid Undulators/wigglers

## Import Radia Insertion Devices

In [1]:
# Add paths to RadiaID and RadiaUtils
import sys
#sys.path.append('../../PyRadia/RadiaID/') # RadiaInsertion Devices
#sys.path.append('../../PyRadia/RadiaUtils/')# Magnetic materials, Radia export functions, etc.

# Import the radia_id module
import radia as rad
import radia_id_mpw_hyb as rid
from jupyter_rs_radia import radia_viewer
import time as tm

## Build and solve a hybrid undulator

In [16]:
build = 'full'
# full, main, side, pole, side_pole, main_pole, main_side

# Main undulator parameters
period = 220 # Undulator period (mm)
gap = 20 # Udulator gap (mm)
#n_periods = 4 # Number of periods
n_poles = 1 # Number of poles

mat = 'ndfeb' # Should be 'cryo', 'ndfeb' or 'smco'

# Build a parameter object according to the material specification
if mat == 'smco':
    # For Sm2Co17 magnets
    params = rid.HybridWigParam(period, n_poles, gap, mag_mat='sm2co17', br=1.1, wig_build=build)
elif mat == 'ndfeb':
    # For room temperature NdFeB (Br may be a bit optimistic)
    params = rid.HybridWigParam(period, n_poles, gap, mag_mat='ndfeb', br=1.29, wig_build=build)
elif mat == 'cryo':
    # For PrFeB magnets at 80 K and Vanadium Permadur poles
    params = rid.HybridWigParam(period, n_poles, gap, mag_mat='ndfeb', br=1.6, pole_mat='fecov', wig_build=build)

# Build and solve
und = rid.HybridWiggler(params)

### Load the object

In [3]:
# Load model exported previously
und = rid.Undulator()
und.load(filename='mpw9cs_g15_' + build, path='')

Error: Para file not found


### Print results

In [7]:
# Print the undulator summary
und.print_wavelength(e=1.2, n=1, theta=0)
# und.print_wavelength()

Wavelength:  23960.960476209617  nm
Energy:  5.1744250040536146e-05 keV
Peak field:  2.385239347615614  T
K:  48.99615553511137
Total length:  300.0  mm


### Export the object

In [8]:
# Export the parameters and model
und.save(filename='mpw' + str(n_poles) + 'cs_g' + str(gap) + '_' + build, path='') # save parameters and model

## 3D plot of the magnet geometry

In [9]:
# Save VTK
und.exportGeometryToVTK('mpw' + str(n_poles) + 'cs_g' + str(gap) + '_' + build)

## Radia Viewer

In [17]:
# set up the radia viewer and display the magnet
rv = radia_viewer.RadiaViewer()
rv.add_geometry('Wiggler', und)
rv.display() 

RuntimeError: Incorrect function arguments: ObjDrwVTK

## Field computations

### Longitudinal field distribution

In [ ]:
und.plot_field(xyz_end=[0, 1000, 0], xyz_start=[0, -1000, 0], n=1000, b='bz', x_axis='d', plot_show=True, plot_title='field_y');

### Transverse field distribution

In [ ]:
und.plot_field([100, 0, 0], [-100, 0, 0], n=1000, b='bz', x_axis='d', plot_show=True, plot_title='field_x');

### Transverse trajectory in logitudinal direction

In [ ]:
und.plot_traj(e=1.2, init_cond=[0, 0, 0, 0], y_range=None, n_points=1000, x_or_z='x', plot_show=True, plot_title='traj_y');

### Field integral

In [ ]:
und.plot_field_int(xyz_end=[400, 0, 0], xyz_start=[-400, 0, 0], dir_int=[0, 1, 0], n=1000, b='bz', x_axis='d', plot_show=True, plot_title='field_int');

### Magnetic force

In [15]:
k = [1, 1, 2]

t0 = tm.time()
und.force2(k)
t1 = tm.time()

print('cpu time(fr):', t1 - t0, 's')

Magnetic forces (fx,fy,fz):  [0.0, 0.0, -13226.944522066333]  Newton
cpu time(fr): 0.8630921840667725 s


## References

[1] The <a href="https://gitlab.esrf.fr/IDM/python/radiaid" target="_blank">radia_id</a> Python module

[2] Radia <a href="https://github.com/ochubar/Radia" target="_blank">Github</a> repository

[3] o Chubar *et al*, J. Synchrotron Rad. 5, pp. 481-484, 1998